In [1]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [66]:
import nltk
from nltk.tokenize import word_tokenize
import markovify

In [41]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [57]:
corpus = []
with open('../data/raw/anki/fra-eng/fra.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        corpus.append(line.lower()) #TODO: Try True-casing instead of lower-casing (frequency-based winner-takes-all)

In [64]:
len(corpus)

189114

In [58]:
%%time
en_fr = [entry.split('\t')[:2] for entry in corpus]
en, fr = zip(*en_fr)
en_tokenized = [word_tokenize(sentence, language='english') for sentence in en]
fr_tokenized = [word_tokenize(sentence, language='french') for sentence in fr]

CPU times: user 42.4 s, sys: 77.9 ms, total: 42.5 s
Wall time: 42.5 s


In [88]:
%%time
en_model = markovify.NewlineText('\n'.join(en), state_size=4)
fr_model = markovify.NewlineText('\n'.join(fr), state_size=4)

CPU times: user 9.99 s, sys: 128 ms, total: 10.1 s
Wall time: 10.1 s


In [100]:
for i in range(5):
    print(en_model.make_sentence(tries=100))

last night my house was robbed while we were away.
i want to go to the supermarket and bought a bag a rice.
they want to tell you who would meet you at the bus stop.
it doesn't matter how much it costs. i'm going to buy you a watch.
last night, we went to the beach together.


In [99]:
for i in range(5):
    print(fr_model.make_sentence(tries=100))

j'ai compris ce que tu voulais me voir.
tu n'es pas obligé de le faire si vous le vouliez vraiment.
j'ai fait des choses dont je ne suis pas surprise que tom ait gagné.
je vous ai dit que vous deviez faire ça ?
il a consacré beaucoup de temps à traiter ce problème.
